In [1]:
!pip install textblob
!pip install tensorflow
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import re
from collections import Counter

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score, silhouette_score
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Dropout, BatchNormalization,Embedding

import warnings
warnings.filterwarnings("ignore")

In [33]:
test_preprocessed_file_path = "../input/dataset-preprocessed-11pm/test_data_preprocessed_11pm_12_12.csv"
train_preprocessed_file_path = "../input/dataset-preprocessed-11pm/train_data_preprocessed_1130pm_12_12.csv"
articles_testing_file_path = "../input/test-original/articles_testing.tsv"

In [4]:
df_preprocessed_data = pd.read_csv(train_preprocessed_file_path)
print(df_preprocessed_data.shape)
print(df_preprocessed_data.describe())
df_preprocessed_data.head()

(97846, 5)
          tag_count
count  97846.000000
mean      19.969033
std        0.507697
min        4.000000
25%       20.000000
50%       20.000000
75%       20.000000
max       23.000000


,content,tags,preprocessed_content,preprocessed_tags,tag_count
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ...","La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,...",trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,...",20
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ...",willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_...",20
2,Giải vô địch ná cao su thế giới năm 2024 đã di...,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng...",giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng...",19
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục...",mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục...",20
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...","toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề...",20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề...",20


In [5]:
df_train = df_preprocessed_data[['preprocessed_content', 'preprocessed_tags']]
print(df_train.shape)
df_train.head()

(97846, 2)


,preprocessed_content,preprocessed_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,..."
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_..."
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng..."
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục..."
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề..."


In [6]:
df_train['preprocessed_tags'] = df_train['preprocessed_tags'].str.split(',')

In [7]:
df_train.head()

,preprocessed_content,preprocessed_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"[la_liga, sevilla, olimpic_lluis_companys, đè_..."
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"[estevao_willian, neymar, giải_vđqg_brazil, se..."
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"[ná_cao_su, giải_vô_địch, giải_ba, thượng_hải,..."
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"[đăng_cai, đại_hội_thể_thao_châu_á, asiad, thể..."
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"[toàn_đoàn, lặn, vũ_đặng_nhật_nam, nguyễn_lê_t..."


In [8]:
all_tags = [tag for tags in df_train['preprocessed_tags'] for tag in tags]
common_tags = [tag for tag, count in Counter(all_tags).most_common(6704)] 

In [9]:
df_train['filtered_tags'] = df_train['preprocessed_tags'].apply(lambda tags: [tag for tag in tags if tag in common_tags])

In [10]:
print(df_train.shape)
df_train.head()

(97846, 3)


,preprocessed_content,preprocessed_tags,filtered_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"[la_liga, sevilla, olimpic_lluis_companys, đè_...","[la_liga, lewandowski, barcelona, raphinha, ha..."
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"[estevao_willian, neymar, giải_vđqg_brazil, se...","[neymar, phá_kỷ_lục, kiến_tạo, gây_sốt, tài_nă..."
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"[ná_cao_su, giải_vô_địch, giải_ba, thượng_hải,...","[giải_vô_địch, giải_ba, thượng_hải, đồng_đội, ..."
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"[đăng_cai, đại_hội_thể_thao_châu_á, asiad, thể...","[đăng_cai, thể_dục, phấn_đấu, chiến_lược_phát_..."
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"[toàn_đoàn, lặn, vũ_đặng_nhật_nam, nguyễn_lê_t...","[toàn_đoàn, đồng_nai, huy_chương_bạc, huy_chươ..."


In [11]:
tags = df_train['filtered_tags'].values
tags = [tag for tag in tags]

In [ ]:
tags

In [12]:
mlb = MultiLabelBinarizer()
mlb.fit(df_train['filtered_tags'])
mlb


MultiLabelBinarizer()

In [13]:
tags_list = mlb.classes_
tags_list

array(["'em", '24k', '2g', ..., 'ứng_trực', 'ứng_tuyển', 'ứng_viên'],
      dtype=object)

In [14]:
y = mlb.transform(df_train['filtered_tags'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
X = df_train['preprocessed_content']
X

0        trận đấu barcelona nhanh_chóng tràn tấn_công v...
1        willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...
2        giải vô_địch ná cao_su thế_giới năm 2024 diễn ...
3        mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...
4        20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...
                               ...                        
97841    động_lực phát_triển bền_vững kinh_tế nhiệm_vụ ...
97842    số_liệu tổng_cục hải_quan 9 đầu năm 2024 xuất_...
97843    mong chiến_lược ứng_phó hiệu_quả thiên_tai kỳ ...
97844    nhà_chức_trách xác_định hoàng_văn_thảo hành_vi...
97845    6 đầu năm 2024 sở y_tế thành_phố đà_nẵng triển...
Name: preprocessed_content, Length: 97846, dtype: object

In [16]:
# train test split protect overfitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(88061, 9785, 88061, 9785)

In [17]:
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

In [18]:
max_content_sample_len = df_train['preprocessed_content'].apply(count_words).max()
print(max_content_sample_len)

8185


In [19]:
X_train.shape,y_train.shape

((88061,), (88061, 6704))

In [20]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train,padding= 'post',maxlen = max_content_sample_len)
X_test = pad_sequences(X_test,padding='post', maxlen = max_content_sample_len)

In [21]:
# cnn model
model_CNN = Sequential()

model_CNN.add(Embedding(len(tokenizer.word_index) + 1,50,input_length=max_content_sample_len))
model_CNN.add(Dropout(0.2))
model_CNN.add(Conv1D(256,3,padding='valid',activation='relu', strides=1))
model_CNN.add(GlobalMaxPooling1D())
model_CNN.add(Dense(256))
model_CNN.add(Dropout(0.2))
model_CNN.add(Activation('relu'))
model_CNN.add(Dense(y_train.shape[1]))
model_CNN.add(Activation('sigmoid'))

model_CNN.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model_CNN.fit(X_train, y_train,batch_size=32,epochs=30,validation_data=(X_test, y_test))


temp_array = np.arange(0.05,0.5,0.05)
scores = []

for probability in temp_array:
    predict_cnn = model_CNN.predict(X_test)
    predict_cnn[predict_cnn>=probability] = 1
    predict_cnn[predict_cnn<probability] = 0
    f1_samples = f1_score(y_test, predict_cnn, average="samples")
    f1_micro = f1_score(y_test, predict_cnn, average="micro")
    f1_macro= f1_score(y_test, predict_cnn, average="macro")
    
    scores.append((f1_samples, f1_micro, f1_macro, probability))
    print("=============================================================")
    print("Accuracy :",metrics.accuracy_score(y_test, predict_cnn))
    print(f"F1_samples: {f1_samples}, F1_micro: {f1_micro}, F1_macro: {f1_macro}, Threshold: {probability}")
    print("Hamming loss :", metrics.hamming_loss(y_test, predict_cnn))
    print("=============================================================")

Epoch 1/30


I0000 00:00:1734051575.536242     111 service.cc:145] XLA service 0x7e322c009d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734051575.537078     111 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734051575.537087     111 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


   3/2752 ━━━━━━━━━━━━━━━━━━━━ 2:04 45ms/step - accuracy: 0.0000e+00 - loss: 0.6921 

I0000 00:00:1734051581.144642     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2752/2752 ━━━━━━━━━━━━━━━━━━━━ 142s 49ms/step - accuracy: 8.3653e-04 - loss: 0.0310 - val_accuracy: 8.1758e-04 - val_loss: 0.0258
Epoch 2/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 131s 47ms/step - accuracy: 0.0046 - loss: 0.0105 - val_accuracy: 0.0258 - val_loss: 0.0109
Epoch 3/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 130s 47ms/step - accuracy: 0.0297 - loss: 0.0083 - val_accuracy: 0.0379 - val_loss: 0.0078
Epoch 4/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 130s 47ms/step - accuracy: 0.0411 - loss: 0.0074 - val_accuracy: 0.0445 - val_loss: 0.0070
Epoch 5/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 130s 47ms/step - accuracy: 0.0489 - loss: 0.0069 - val_accuracy: 0.0490 - val_loss: 0.0066
Epoch 6/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 129s 47ms/step - accuracy: 0.0512 - loss: 0.0066 - val_accuracy: 0.0559 - val_loss: 0.0064
Epoch 7/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 130s 47ms/step - accuracy: 0.0571 - loss: 0.0063 - val_accuracy: 0.0567 - val_loss: 0.0062
Epoch 8/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 130s 47ms/step - accuracy: 0.0

In [22]:
model_CNN.save('cnn_model.h5')

In [38]:
df_test = pd.read_csv(test_preprocessed_file_path)

In [39]:
max_content_test_sample_len = df_test['preprocessed_content'].apply(count_words).max()
print(max_content_test_sample_len)

1047


In [ ]:
tags_predict_list = []
num = 0
for content in df_test['preprocessed_content']:
    num+=1
    print(num)
    sequence = tokenizer.texts_to_sequences([content])
    padded_sequence = pad_sequences(sequence, padding='post', maxlen=max_content_test_sample_len)
    prediction = model_CNN.predict(padded_sequence)
    prediction = prediction.flatten()
    prediction_threshold = 0.15
    predicted_tags = mlb.classes_[prediction >= prediction_threshold]
    tags_predict = ",".join(tag.replace("_", " ") for tag in predicted_tags)
    tags_predict_list.append(tags_predict)

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
25
1/1 ━━━━━━━━━━━━━━━━━━

In [43]:
df_result = pd.read_csv(articles_testing_file_path, sep='\t')
df_result['tags'] = tags_predict_list

In [44]:
df_result.head()

,content,tags
0,"Vào tối 22/10, sau đêm bán kết của cuộc thi Ho...","hoa hậu,miss grand international,miss grand in..."
1,"Tân Hoa Xã đưa tin rằng vào ngày 9/10, phát ng...","kiev,liên bang nga,moscow,nato,phương tây,quân..."
2,Chuyến thăm của Lloyd Austin diễn ra chỉ vài n...,"austin,bầu cử,bộ trưởng quốc phòng,chuyến thăm..."
3,"Trong dự thảo Luật Nhà giáo, Bộ GD&ĐT đã đưa r...","bộ gd,bộ giáo dục và đào tạo,dự thảo,dự thảo l..."
4,"Trong buổi gặp gỡ, Chủ tịch Hà Thị Nga đã bày ...","chuyến thăm,chúc mừng,chủ tịch nước,cuba,cấp c..."


In [46]:
df_result.to_csv('cnn_result.tsv', sep='\t', index=False)